# Testing Scaling limits of GTSAM on BAL dataset

![image](media/balRef.png)

The dataset can be found at: https://grail.cs.washington.edu/projects/bal/

In [1]:
# This notebook tests GTSAM BA performance on BAL format datasets

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import gc
import gtsam
import pickle
import os
from helper_bal import (
    load_bal_data,
    build_factor_graph,
    run_bundle_adjustment,
    print_results,
    get_schur_sparsity,
    save_results,
    load_results,
    plot_reconstruction,
    check_memory_available,
    estimate_memory_requirement,
    C, P  # Symbol shortcuts
)

In [2]:
def check_memory_status():
    """Print current memory status."""
    import psutil
    mem = psutil.virtual_memory()
    print(f"\nMemory Status:")
    print(f"  Total: {mem.total / (1024**3):.1f} GB")
    print(f"  Available: {mem.available / (1024**3):.1f} GB")
    print(f"  Used: {mem.used / (1024**3):.1f} GB ({mem.percent}%)")
    print(f"  Free: {mem.free / (1024**3):.1f} GB")


def force_cleanup():
    """Force garbage collection and print memory freed."""
    import psutil
    
    before = psutil.virtual_memory().used / (1024**3)
    gc.collect()
    after = psutil.virtual_memory().used / (1024**3)
    freed = before - after
    
    print(f"\nGarbage collection completed")
    if freed > 0.01:
        print(f"  Freed: {freed:.2f} GB")
    check_memory_status()


In [3]:
# %% Configuration
RESULTS_FILE = 'bal_results.pkl'
RESULTS_CSV = 'bal_scaling_results.csv'
PLOT_DIR = 'plots'

# Create plot directory
os.makedirs(PLOT_DIR, exist_ok=True)

In [4]:
all_results = load_results(RESULTS_FILE)
if all_results:
    print(f"Loaded {len(all_results)} existing results from {RESULTS_FILE}")
    print(f"Completed datasets: {[r['name'] for r in all_results]}")
else:
    print("No existing results found. Starting fresh.")
    all_results = []


No existing results found. Starting fresh.


In [5]:
# Dataset paths - update with your actual BAL file paths
DATASET_PATHS = [
    ('Final-93-61203', '/home/anatharv1/GTSAM-Project/BA_datasets/bal/Final/problem-93-61203-pre.txt'),
    ('Final-394-100368', '/home/anatharv1/GTSAM-Project/BA_datasets/bal/Final/problem-394-100368-pre.txt'),
    ('Final-871-527480', '/home/anatharv1/GTSAM-Project/BA_datasets/bal/Final/problem-871-527480-pre.txt'),
    ('Final-961-187103', '/home/anatharv1/GTSAM-Project/BA_datasets/bal/Final/problem-961-187103-pre.txt'),
    ('Final-1936-649673', '/home/anatharv1/GTSAM-Project/BA_datasets/bal/Final/problem-1936-649673-pre.txt'),
    ('Final-3068-310854', '/home/anatharv1/GTSAM-Project/BA_datasets/bal/Final/problem-3068-310854-pre.txt'),
    ('Final-4585-1324582', '/home/anatharv1/GTSAM-Project/BA_datasets/bal/Final/problem-4585-1324582-pre.txt'),
    ('Final-13682-4456117', '/home/anatharv1/GTSAM-Project/BA_datasets/bal/Final/problem-13682-4456117-pre.txt'),    
]


In [6]:
def run_single_dataset(idx, verbose=True, plot=True, max_points=10000,
                        lambda_initial=1.0, lambda_upper_bound=1e9):
    """
    Run bundle adjustment on a single dataset with aggressive memory management.
    
    Args:
        idx: dataset index
        verbose: print detailed output
        plot: if True, generate visualization before cleanup
        max_points: maximum number of points to plot
        lambda_initial: initial damping parameter (default: 1.0)
        lambda_upper_bound: maximum lambda value (default: 1e9, increase for difficult problems)
    """
    global all_results
    
    name, bal_file = DATASET_PATHS[idx]
    
    # Check if already completed
    completed_names = [r['name'] for r in all_results]
    if name in completed_names:
        print(f"\n{'='*70}")
        print(f"DATASET {idx+1}: {name} - ALREADY COMPLETED, SKIPPING")
        print(f"{'='*70}")
        return True
    
    # Force garbage collection before starting
    gc.collect()
    
    print(f"\n{'='*70}")
    print(f"DATASET {idx+1}: {name}")
    print(f"{'='*70}")
    
    try:
        from helper_bal import (
            load_bal_data,
            build_factor_graph,
            run_bundle_adjustment,
            print_results,
            plot_reconstruction
        )
        
        # Load data
        scene_data = load_bal_data(bal_file)
        
        # Build graph
        graph, initial = build_factor_graph(scene_data)
        
        # Run optimization (always LM) with custom lambda settings
        result, metrics = run_bundle_adjustment(
            graph, initial, 
            max_iterations=50,
            verbose=verbose,
            lambda_initial=lambda_initial,
            lambda_upper_bound=lambda_upper_bound
        )
        
        # Print results
        print_results(name, scene_data, metrics)
        
        # Count observations
        total_obs = sum(scene_data.track(j).numberMeasurements() 
                       for j in range(scene_data.numberTracks()))
        
        # Store results
        result_entry = {
            'name': name,
            'num_cameras': scene_data.numberCameras(),
            'num_points': scene_data.numberTracks(),
            'num_observations': total_obs,
            **metrics
        }
        all_results.append(result_entry)
        
        # Save immediately
        save_results(all_results, RESULTS_FILE)
        save_results_csv(all_results, RESULTS_CSV)
        
        # Plot BEFORE cleanup (so we don't re-optimize)
        if plot:
            print("\n" + "="*70)
            print("GENERATING VISUALIZATION")
            print("="*70)
            try:
                plot_reconstruction(
                    scene_data, 
                    initial, 
                    result, 
                    name,
                    max_points=max_points,
                    save_path=f"{name}_reconstruction.html"
                )
                print(f"✓ Plot saved to: {name}_reconstruction.html")
            except Exception as e:
                print(f"  Plotting failed: {e}")
                import traceback
                traceback.print_exc()
        
        # Aggressive cleanup
        del scene_data, graph, initial, result
        gc.collect()
        print(f"\n  Memory cleaned up")
        
        return True
        
    except Exception as e:
        print(f"ERROR: {type(e).__name__}: {e}")
        import traceback
        traceback.print_exc()
        
        # Save what we have
        save_results(all_results, RESULTS_FILE)
        
        # Force cleanup
        gc.collect()
        
        return False


def plot_dataset(idx, max_points=10000):
    """
    Plot reconstruction for a dataset that was run without plotting.
    This will re-run the optimization.
    
    Args:
        idx: dataset index
        max_points: maximum number of points to plot
    """
    global all_results
    
    name, bal_file = DATASET_PATHS[idx]
    
    # Check if this dataset has been completed
    completed_names = [r['name'] for r in all_results]
    if name not in completed_names:
        print(f"Dataset {name} has not been run yet!")
        print(f"Run it first with: run_single_dataset({idx})")
        return
    
    print(f"\nPlotting reconstruction for: {name}")
    print("  WARNING: This will re-run the optimization")
    
    try:
        from helper_bal import (
            load_bal_data,
            build_factor_graph,
            run_bundle_adjustment,
            plot_reconstruction
        )
        
        # Reload and optimize
        print("  Loading data...")
        scene_data = load_bal_data(bal_file)
        
        print("  Building graph...")
        graph, initial = build_factor_graph(scene_data)
        
        print("  Re-running optimization...")
        result, metrics = run_bundle_adjustment(
            graph, initial, 
            max_iterations=50,
            verbose=False
        )
        
        print("  Generating plot...")
        plot_reconstruction(
            scene_data, 
            initial, 
            result, 
            name,
            max_points=max_points,
            save_path=f"{name}_reconstruction.html"
        )
        
        # Cleanup
        del scene_data, graph, initial, result
        gc.collect()
        
        print(f"\n✓ Plot saved to: {name}_reconstruction.html")
        
    except Exception as e:
        print(f"ERROR: {e}")
        import traceback
        traceback.print_exc()

In [7]:
def save_results_csv(results, filename):
    """Save results to CSV file."""
    if not results:
        return
    
    # Create a simplified version for CSV (no lists)
    csv_data = []
    for r in results:
        csv_entry = {k: v for k, v in r.items() 
                    if not isinstance(v, (list, np.ndarray))}
        csv_data.append(csv_entry)
    
    df = pd.DataFrame(csv_data)
    df.to_csv(filename, index=False)
    print(f"  CSV saved to {filename}")

In [8]:
check_memory_status()
run_single_dataset(0, verbose=True)  
force_cleanup()


Memory Status:
  Total: 31.1 GB
  Available: 27.6 GB
  Used: 3.0 GB (11.2%)
  Free: 24.7 GB

DATASET 1: Final-93-61203
  Loading BAL file: /home/anatharv1/GTSAM-Project/BA_datasets/bal/Final/problem-93-61203-pre.txt
  Loaded 93 cameras, 61203 tracks
Building factor graph...
  93 cameras, 61203 tracks, 287451 observations
  Graph size: 287453 factors
  Values size: 61296 variables
  Initial error: 5.65e+06

LM Parameters:
  Max iterations: 50
  Initial lambda: 1.0
  Lambda factor: 10.0
  Lambda upper bound: 1000000000.0
  Lambda lower bound: 0.0
  Relative error tol: 1e-05
  Absolute error tol: 1e-05

Running Bundle Adjustment...
  Initial error: 5.646907e+06
------------------------------------------------------------
Initial error: 5.64691e+06, values: 61296
iter      cost      cost_change    lambda  success iter_time
   0  1.628755e+05    5.48e+06    1.00e+00     1    1.84e+00
  Iter   1: error = 1.628755e+05, reduction =  97.116%, lambda = 1.00e-01, time = 2.10s
   1  1.461321e+05 

✓ Plot saved to: Final-93-61203_reconstruction.html

  Memory cleaned up

Garbage collection completed

Memory Status:
  Total: 31.1 GB
  Available: 24.9 GB
  Used: 5.8 GB (20.1%)
  Free: 21.7 GB


In [9]:
check_memory_status()
run_single_dataset(1, verbose=True)  
force_cleanup()


Memory Status:
  Total: 31.1 GB
  Available: 24.9 GB
  Used: 5.8 GB (20.1%)
  Free: 21.7 GB

DATASET 2: Final-394-100368
  Loading BAL file: /home/anatharv1/GTSAM-Project/BA_datasets/bal/Final/problem-394-100368-pre.txt
  Loaded 394 cameras, 100368 tracks
Building factor graph...
  394 cameras, 100368 tracks, 534408 observations
  Graph size: 534410 factors
  Values size: 100762 variables
  Initial error: 4.55e+06

LM Parameters:
  Max iterations: 50
  Initial lambda: 1.0
  Lambda factor: 10.0
  Lambda upper bound: 1000000000.0
  Lambda lower bound: 0.0
  Relative error tol: 1e-05
  Absolute error tol: 1e-05

Running Bundle Adjustment...
  Initial error: 4.545733e+06
------------------------------------------------------------
Initial error: 4.54573e+06, values: 100762
  Iter   1: error = 3.522262e+05, reduction =  92.251%, lambda = 1.00e-01, time = 4.95s
iter      cost      cost_change    lambda  success iter_time
   0  3.522262e+05    4.19e+06    1.00e+00     1    4.56e+00
   1  3.0

✓ Plot saved to: Final-394-100368_reconstruction.html

  Memory cleaned up

Garbage collection completed

Memory Status:
  Total: 31.1 GB
  Available: 22.6 GB
  Used: 8.0 GB (27.4%)
  Free: 19.4 GB


In [10]:
check_memory_status()
run_single_dataset(2, verbose=True)  
force_cleanup()


Memory Status:
  Total: 31.1 GB
  Available: 22.6 GB
  Used: 8.0 GB (27.4%)
  Free: 19.4 GB

DATASET 3: Final-871-527480
  Loading BAL file: /home/anatharv1/GTSAM-Project/BA_datasets/bal/Final/problem-871-527480-pre.txt
  Loaded 871 cameras, 527480 tracks
Building factor graph...
  871 cameras, 527480 tracks, 2785977 observations
  Graph size: 2785979 factors
  Values size: 528351 variables
  Initial error: 1.05e+08

LM Parameters:
  Max iterations: 50
  Initial lambda: 1.0
  Lambda factor: 10.0
  Lambda upper bound: 1000000000.0
  Lambda lower bound: 0.0
  Relative error tol: 1e-05
  Absolute error tol: 1e-05

Running Bundle Adjustment...
  Initial error: 1.046181e+08
------------------------------------------------------------
  Iter   1: error = 2.313852e+06, reduction =  97.788%, lambda = 1.00e-01, time = 23.24s
Initial error: 1.04618e+08, values: 528351
  Iter   2: error = 1.873697e+06, reduction =  19.023%, lambda = 1.00e+00, time = 61.62s
iter      cost      cost_change    lamb

✓ Plot saved to: Final-871-527480_reconstruction.html

  Memory cleaned up

Garbage collection completed

Memory Status:
  Total: 31.1 GB
  Available: 14.1 GB
  Used: 16.5 GB (54.8%)
  Free: 10.7 GB


In [11]:
# check_memory_status()
# run_single_dataset(3, verbose=True, plot=True, 
#                    lambda_upper_bound=1e12)  # 1000x larger
# force_cleanup()


####  Logs from cell before Kernel stops due to lack of RAM

Memory Status:
  Total: 31.1 GB

  Available: 27.9 GB
  
  Used: 2.7 GB (10.4%)
  
  Free: 26.5 GB

DATASET 4: Final-961-187103

  
  Loading BAL file: /home/anatharv1/GTSAM-Project/BA_datasets/bal/Final/problem-961-187103-pre.txt
  
  Loaded 961 cameras, 187103 tracks

Building factor graph...

  961 cameras, 187103 tracks, 1692975 observations

  Graph size: 1692977 factors

  Values size: 188064 variables

  Initial error: 4.32e+07

In [12]:
# check_memory_status()
# run_single_dataset(4, verbose=True, plot=True, lambda_initial=1e4,
#                    lambda_upper_bound=1e12)  # 1000x larger
# force_cleanup()

WARNING: Large dataset - monitor memory usage!

Memory Status:
  Total: 31.1 GB
  Available: 28.4 GB
  Used: 2.2 GB (8.6%)
  Free: 28.1 GB


DATASET 5: Final-1936-649673

  Loading BAL file: /home/anatharv1/GTSAM-Project/BA_datasets/bal/Final/problem-1936-649673-pre.txt
  Loaded 1936 cameras, 649673 tracks
Building factor graph...
  1936 cameras, 649673 tracks, 5213733 observations
  Graph size: 5213735 factors
  Values size: 651609 variables
  Initial error: 1.83e+08

In [13]:
print("WARNING: Large dataset - monitor memory usage!")
check_memory_status()
run_single_dataset(5, verbose=True, plot=True, lambda_initial=1e4,
                   lambda_upper_bound=1e15)  # 1000x larger
force_cleanup()


Memory Status:
  Total: 31.1 GB
  Available: 14.1 GB
  Used: 16.5 GB (54.8%)
  Free: 10.7 GB

DATASET 6: Final-3068-310854
  Loading BAL file: /home/anatharv1/GTSAM-Project/BA_datasets/bal/Final/problem-3068-310854-pre.txt
  Loaded 3068 cameras, 310854 tracks
Building factor graph...
  3068 cameras, 310854 tracks, 1653812 observations
  Graph size: 1653814 factors
  Values size: 313922 variables
  Initial error: 9.05e+07

LM Parameters:
  Max iterations: 50
  Initial lambda: 10000.0
  Lambda factor: 10.0
  Lambda upper bound: 1000000000000000.0
  Lambda lower bound: 0.0
  Relative error tol: 1e-05
  Absolute error tol: 1e-05

Running Bundle Adjustment...
  Initial error: 9.047461e+07
------------------------------------------------------------
Initial error: 9.04746e+07, values: 313922
iter      cost      cost_change    lambda  success iter_time
   0      inf    4.94e-324    1.00e+04     0    5.29e+01
iter      cost      cost_change    lambda  success iter_time
   0      inf    4.94e-

✓ Plot saved to: Final-3068-310854_reconstruction.html

  Memory cleaned up

Garbage collection completed

Memory Status:
  Total: 31.1 GB
  Available: 14.0 GB
  Used: 16.6 GB (54.9%)
  Free: 10.6 GB


In [14]:
# print("WARNING: Large dataset - monitor memory usage!")
# check_memory_status()
# run_single_dataset(6, verbose=True, plot=True, lambda_initial=1e4,
#                    lambda_upper_bound=1e15)  # 1000x larger
# force_cleanup()

WARNING: Large dataset - monitor memory usage!

Memory Status:
  Total: 31.1 GB
  Available: 19.8 GB
  Used: 10.8 GB (36.2%)
  Free: 18.7 GB

======================================================================
DATASET 7: Final-4585-1324582
======================================================================
  Loading BAL file: /home/anatharv1/GTSAM-Project/BA_datasets/bal/Final/problem-4585-1324582-pre.txt
  Loaded 4585 cameras, 1324582 tracks
Building factor graph...
  4585 cameras, 1324582 tracks, 9125125 observations
  Graph size: 9125127 factors
  Values size: 1329167 variables
  Initial error: 6.03e+08

In [15]:
# print("WARNING: Large dataset - monitor memory usage!")
# check_memory_status()
# run_single_dataset(7, verbose=True, plot=True, lambda_initial=1e4,
#                    lambda_upper_bound=1e15)  # 1000x larger
# force_cleanup()

In [16]:

def generate_summary_report():
    """Generate comprehensive summary report with visualizations."""
    global all_results
    
    if not all_results:
        print("No results available! Run some datasets first.")
        return
    
    import pandas as pd
    import numpy as np
    try:
        import plotly.graph_objects as go
        from plotly.subplots import make_subplots
        use_plotly = True
    except ImportError:
        print("Plotly not available, summary table only")
        use_plotly = False
    
    # Create summary dataframe
    summary_df = pd.DataFrame([
        {
            'Dataset': r['name'],
            'Cameras': r['num_cameras'],
            'Points': r['num_points'],
            'Observations': r['num_observations'],
            'Time (s)': f"{r['time']:.1f}",
            'Max CPU (%)': f"{r['max_cpu']:.1f}",
            'Avg CPU (%)': f"{r['avg_cpu']:.1f}",
            'Max RAM (GB)': f"{r['max_ram_gb']:.2f}",
            'Avg RAM (GB)': f"{r['avg_ram_gb']:.2f}",
            'Initial Error': f"{r['initial_error']:.2e}",
            'Final Error': f"{r['final_error']:.2e}",
            'Iterations': r['iterations'],
            'Success': '✓' if r['success'] else '✗'
        }
        for r in all_results
    ])
    
    print("\n" + "="*100)
    print("SUMMARY OF ALL BUNDLE ADJUSTMENT RUNS")
    print("="*100)
    print(summary_df.to_string(index=False))
    print("="*100)
    
    if not use_plotly:
        return summary_df
    
    # Create interactive Plotly visualizations
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Optimization Time vs Problem Size',
            'Peak Memory Usage vs Problem Size',
            'CPU Utilization',
            'Optimization Error Reduction'
        ),
        specs=[[{'type': 'scatter'}, {'type': 'scatter'}],
               [{'type': 'bar'}, {'type': 'bar'}]]
    )
    
    # Plot 1: Time vs Problem Size
    fig.add_trace(
        go.Scatter(
            x=[r['num_cameras'] for r in all_results],
            y=[r['time'] for r in all_results],
            mode='markers',
            marker=dict(size=12, color='steelblue', opacity=0.7),
            text=[r['name'] for r in all_results],
            hovertemplate='<b>%{text}</b><br>Cameras: %{x}<br>Time: %{y:.1f}s<extra></extra>',
            name='Time'
        ),
        row=1, col=1
    )
    fig.update_xaxes(title_text="Number of Cameras", row=1, col=1)
    fig.update_yaxes(title_text="Time (seconds)", row=1, col=1)
    
    # Plot 2: Max RAM vs Problem Size
    fig.add_trace(
        go.Scatter(
            x=[r['num_cameras'] for r in all_results],
            y=[r['max_ram_gb'] for r in all_results],
            mode='markers',
            marker=dict(size=12, color='coral', opacity=0.7),
            text=[r['name'] for r in all_results],
            hovertemplate='<b>%{text}</b><br>Cameras: %{x}<br>RAM: %{y:.2f} GB<extra></extra>',
            name='RAM'
        ),
        row=1, col=2
    )
    fig.update_xaxes(title_text="Number of Cameras", row=1, col=2)
    fig.update_yaxes(title_text="Max RAM (GB)", row=1, col=2)
    
    # Plot 3: CPU Usage
    names = [r['name'] for r in all_results]
    max_cpu = [r['max_cpu'] for r in all_results]
    avg_cpu = [r['avg_cpu'] for r in all_results]
    
    fig.add_trace(
        go.Bar(
            x=names,
            y=max_cpu,
            name='Max CPU',
            marker_color='steelblue',
            opacity=0.7,
            hovertemplate='<b>%{x}</b><br>Max CPU: %{y:.1f}%<extra></extra>'
        ),
        row=2, col=1
    )
    fig.add_trace(
        go.Bar(
            x=names,
            y=avg_cpu,
            name='Avg CPU',
            marker_color='lightblue',
            opacity=0.7,
            hovertemplate='<b>%{x}</b><br>Avg CPU: %{y:.1f}%<extra></extra>'
        ),
        row=2, col=1
    )
    fig.update_xaxes(title_text="Dataset", tickangle=45, row=2, col=1)
    fig.update_yaxes(title_text="CPU Usage (%)", row=2, col=1)
    
    # Plot 4: Error Reduction
    successful = [r for r in all_results if r['success']]
    if successful:
        error_reduction = [(1 - r['final_error']/r['initial_error'])*100 for r in successful]
        names_success = [r['name'] for r in successful]
        
        fig.add_trace(
            go.Bar(
                x=names_success,
                y=error_reduction,
                marker_color='lightgreen',
                opacity=0.7,
                hovertemplate='<b>%{x}</b><br>Error Reduction: %{y:.2f}%<extra></extra>',
                showlegend=False
            ),
            row=2, col=2
        )
        fig.update_xaxes(title_text="Dataset", tickangle=45, row=2, col=2)
        fig.update_yaxes(title_text="Error Reduction (%)", row=2, col=2)
    
    # Update layout
    fig.update_layout(
        title_text="GTSAM Bundle Adjustment - Scaling Analysis",
        height=800,
        showlegend=True,
        legend=dict(x=0.5, y=-0.15, xanchor='center', orientation='h')
    )
    
    # Show the plot
    fig.show()
    
    # Save as HTML
    fig.write_html("bal_scaling_summary.html")
    print(f"\n✓ Interactive summary plot saved to: bal_scaling_summary.html")
    
    return summary_df


def print_comparison_table():
    """Print a detailed comparison table of all runs."""
    global all_results
    
    if not all_results:
        print("No results available!")
        return
    
    import pandas as pd
    
    # Detailed comparison
    comparison_df = pd.DataFrame([
        {
            'Dataset': r['name'],
            'Cameras': r['num_cameras'],
            'Points': r['num_points'],
            'Obs': r['num_observations'],
            'Time': r['time'],
            'Iters': r['iterations'],
            'Init Error': r['initial_error'],
            'Final Error': r['final_error'],
            'Reduction %': (1 - r['final_error']/r['initial_error'])*100,
            'Max RAM': r['max_ram_gb'],
            'Success': r['success']
        }
        for r in all_results
    ])
    
    print("\n" + "="*120)
    print("DETAILED COMPARISON TABLE")
    print("="*120)
    print(comparison_df.to_string(index=False))
    print("="*120)
    
    # Summary statistics
    print("\nSUMMARY STATISTICS:")
    print(f"  Total datasets completed: {len(all_results)}")
    print(f"  Total cameras: {comparison_df['Cameras'].sum():,}")
    print(f"  Total points: {comparison_df['Points'].sum():,}")
    print(f"  Total observations: {comparison_df['Obs'].sum():,}")
    print(f"  Total time: {comparison_df['Time'].sum():.1f} seconds ({comparison_df['Time'].sum()/60:.1f} minutes)")
    print(f"  Average time per dataset: {comparison_df['Time'].mean():.1f} seconds")
    print(f"  Max RAM usage: {comparison_df['Max RAM'].max():.2f} GB")
    print(f"  Average RAM usage: {comparison_df['Max RAM'].mean():.2f} GB")
    print(f"  Average error reduction: {comparison_df['Reduction %'].mean():.2f}%")
    print(f"  Success rate: {comparison_df['Success'].sum()}/{len(all_results)}")
    
    return comparison_df


In [17]:
summary_df = generate_summary_report()
comparison_df = print_comparison_table()


SUMMARY OF ALL BUNDLE ADJUSTMENT RUNS
          Dataset  Cameras  Points  Observations Time (s) Max CPU (%) Avg CPU (%) Max RAM (GB) Avg RAM (GB) Initial Error Final Error  Iterations Success
   Final-93-61203       93   61203        287451      8.2         5.6         5.0         2.80         2.79      5.65e+06    1.46e+05           5       ✓
 Final-394-100368      394  100368        534408    147.4         5.0         4.3         4.92         4.92      4.55e+06    3.00e+05          16       ✓
 Final-871-527480      871  527480       2785977    814.2         4.1         4.0        13.33        13.19      1.05e+08    1.81e+06          19       ✓
Final-3068-310854     3068  310854       1653812   3432.0         4.0         4.0        12.88        12.88      9.05e+07    4.17e+06          20       ✓



✓ Interactive summary plot saved to: bal_scaling_summary.html

DETAILED COMPARISON TABLE
          Dataset  Cameras  Points     Obs        Time  Iters   Init Error  Final Error  Reduction %   Max RAM  Success
   Final-93-61203       93   61203  287451    8.203256      5 5.646907e+06 1.460842e+05    97.413023  2.796146     True
 Final-394-100368      394  100368  534408  147.423919     16 4.545733e+06 3.001917e+05    93.396188  4.923401     True
 Final-871-527480      871  527480 2785977  814.169960     19 1.046181e+08 1.814949e+06    98.265168 13.332783     True
Final-3068-310854     3068  310854 1653812 3431.969776     20 9.047461e+07 4.174527e+06    95.385969 12.884636     True

SUMMARY STATISTICS:
  Total datasets completed: 4
  Total cameras: 4,426
  Total points: 999,905
  Total observations: 5,261,648
  Total time: 4401.8 seconds (73.4 minutes)
  Average time per dataset: 1100.4 seconds
  Max RAM usage: 13.33 GB
  Average RAM usage: 8.48 GB
  Average error reduction: 96.12%
  Su